## Z-Spread 

Author: Will Carpenter

Date:   April 6th, 2024

A comprehensive Python notebook to create a spot interest rate curve and calculate bond Z-spreads, among other pricing metrics. This notebook leverages custom Python modules (see importing and project folder for all source code).

In [4]:
# import
import pandas as pd
import numpy as np
# Custom modules
import mortgage_cash_flow as mbs
import bond_price as px
import spot_rate_bootstrap as boot 
import z_spread as pricing

Obtain U.S. Treasury Par-Yield Data

In [6]:
# Importing data available on Github  
tsy  = pd.read_csv("https://raw.githubusercontent.com/wrcarpenter/Z-Spread/main/Data/daily-treasury-rates.csv", header=0)
head = pd.read_csv("https://raw.githubusercontent.com/wrcarpenter/Z-Spread/main/Data/daily-treasury-spot-header.csv")
# Add charting here

Interpolate Semi-Annual Par Yield Rates

In [ ]:
ylds = boot.interpolate_yields(tsy, head)

Boostrap Treasury Yield Data to Create Spot Rate Curve

In [ ]:
spots         = boot.spot_rate_bootstrap(ylds, tsy, head)
spots_monthly = boot.spot_rates_monthly(spots)